In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame, read_sql_query
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 4568}


4568

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 8045}


8045

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

beneficiare_direct.case_id.count()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,8fd7f28e6c31484993c3153565feb429,cmariedanielle,2022-03-08T14:13:12.374000Z,1e237732-aca9-485c-acd3-b7d285559838,4,Darius,Lejean
1,8fd7f28e6c31484993c3153565feb429,cmariedanielle,2022-03-08T14:09:45.420000Z,ad3ea0ae-a576-449f-8a8e-98093bb4a035,1,Avrius,Ronald
2,8fd7f28e6c31484993c3153565feb429,cmariedanielle,2022-03-08T14:01:30.742000Z,feae7d183782433a9d6c56a690aaaa13,0,Holte,Bel Fleur
3,8fd7f28e6c31484993c3153565feb429,cmariedanielle,2022-03-08T14:00:45.447000Z,4cbc860426a44bd084c71a7bf9900caf,4,Bien Aime,Fritzson
4,8fd7f28e6c31484993c3153565feb429,cmariedanielle,2022-03-08T13:57:45.159000Z,ab321b1ca6a245bbb32ebc3bd0422ee5,2,Romelus,Berge
...,...,...,...,...,...,...,...
4419,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
4420,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
4421,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
4422,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


4424

In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect
beneficiare_indirect.ib_case_id.count()



,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
8040,8fd7f28e6c31484993c3153565feb429,28093bdc-629f-4335-ac99-7dfcd19225bb,2,1,non,non,ad3ea0ae-a576-449f-8a8e-98093bb4a035,child
8041,8fd7f28e6c31484993c3153565feb429,384edd9c-a89f-471a-9bc8-9b6c78e0b115,7,1,non,non,1e237732-aca9-485c-acd3-b7d285559838,child
8042,8fd7f28e6c31484993c3153565feb429,1dfc49ab-4a12-4bb0-8bbd-1f2188a11e67,2,1,non,non,1e237732-aca9-485c-acd3-b7d285559838,child
8043,8fd7f28e6c31484993c3153565feb429,eecd31c5-4eb4-4b3b-afcb-417dc6f4def5,16,1,non,non,1e237732-aca9-485c-acd3-b7d285559838,child


8045

In [7]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query = f'''
SELECT
	au.username,
	au.email,
	count(*) as qty
FROM
	muso_household_2022 mh
	LEFT JOIN auth_users au ON mh.created_by = au.id
	group by au.username, mh.id_patient
'''

muso_hiv = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()


True

In [14]:
result_hivmuso = muso_hiv.groupby(['username','email']).count()
result_hivmuso.loc['Total']= result_hivmuso.sum(numeric_only=True, axis=0)
result_hivmuso.reset_index(inplace=True)
result_hivmuso
result_hivmuso.qty.sum()

,index,qty
0,"(Ernstbelabre, ernst.belabre@carisfoundationin...",6
1,"(Gustave, firma.gustave@carisfoundationintl.org)",45
2,"(June007, jeanrony.valbrun@carisfoundationintl...",16
3,"(jfodney17, fodney.joseph@carisfoundationintl....",18
4,"(josy22, josy.serafils@carisfoundationintl.org)",21
5,"(judesonvOLMAR, judeson.volmar@carisfoundation...",27
6,"(marcakernstdubois, kernst.dubois@carisfoundat...",3
7,"(suzanne, suzanne.clerge@carisfoundationintl.org)",116
8,Total,252


504

In [11]:
total = DataFrame(
    {
        "total":[result_hivmuso.qty.sum()]
    }
)

total

,total
0,252
